In [1]:
from sys import argv
from pymongo import MongoClient
import json

# client = MongoClient('127.0.0.1', 27017)
client = MongoClient('mongodb://127.0.0.1:27017/')
db = client.movielens

In [2]:
db_list = client.list_database_names()
print(db_list)

['admin', 'config', 'lab', 'local', 'movielens', 'mydatabase', 'mydb', 'test']


In [3]:
collections = db.list_collection_names()

print(collections)

['tags', 'movies', 'ratings']


In [4]:
# 데이터베이스 내의 모든 컬렉션 이름 가져오기
collections = db.list_collection_names()

# 각 컬렉션에 대해 도큐먼트 5개 조회 및 전체 도큐먼트 수 출력
for collection_name in collections:
    collection = db[collection_name]
    print(f"--- {collection_name} ---")
    # 컬렉션 내의 전체 도큐먼트 수
    total_documents = collection.count_documents({})
    print(f"Total documents: {total_documents}")
    # 컬렉션 내의 첫 5개 도큐먼트 조회
    for document in collection.find().limit(5):
        print(document)
    print("\n")


--- tags ---
Total documents: 1093360
{'_id': ObjectId('64196f7a8d4cb3f0f36bbb74'), 'userId': 19, 'movieId': 7099, 'tag': 'ecology', 'timestamp': 1445286153}
{'_id': ObjectId('64196f7a8d4cb3f0f36bbb79'), 'userId': 91, 'movieId': 590, 'tag': 'American Civil War', 'timestamp': 1414248567}
{'_id': ObjectId('64196f7a8d4cb3f0f36bbb77'), 'userId': 87, 'movieId': 109487, 'tag': 'space travel', 'timestamp': 1522676685}
{'_id': ObjectId('64196f7a8d4cb3f0f36bbb7a'), 'userId': 91, 'movieId': 1719, 'tag': 'wintry', 'timestamp': 1414932700}
{'_id': ObjectId('64196f7a8d4cb3f0f36bbb7b'), 'userId': 91, 'movieId': 1962, 'tag': 'aging', 'timestamp': 1414703813}


--- movies ---
Total documents: 62423
{'_id': ObjectId('64196f417e2562c864eb19c4'), 'movieId': 166, 'title': 'Doom Generation, The (1995)', 'genres': ['Comedy', 'Crime', 'Drama']}
{'_id': ObjectId('64196f417e2562c864eb19c6'), 'movieId': 169, 'title': 'Free Willy 2: The Adventure Home (1995)', 'genres': ['Adventure', 'Children', 'Drama']}
{'_id'

In [33]:
def q2(input_tag: str):
    # TODO: 
    mov_cursor = db.tags.find({'tag': input_tag}, {'movieId': 1, '_id': 0})
    mov_list = set([m['movieId'] for m in mov_cursor])
    if not mov_list:
        print(f"No title found with tag: {input_tag}")
        return          
    titles = set()
    for mov in mov_list:
        title = db.movies.find_one({ 'movieId': mov}, {'title':1, '_id':0})
        titles.add(title['title'])
  
    for title in sorted(titles):
        print(title)
        
        

q2(input_tag = 'ecology')

180° South (180 Degrees South) (180° South: Conquerors of the Useless) (2010)
A River of Waste: The Hazardous Truth About Factory Farms (2009)
All Watched Over by Machines of Loving Grace (2011)
An Inconvenient Sequel (2017)
Autómata (Automata) (2014)
Avatar (2009)
Chasing Coral (2017)
Earth (2007)
El Libro de Lila (2018)
Fly Away Home (1996)
Godzilla and Mothra: The Battle for Earth (Gojira vs. Mosura) (1992)
Grizzly Man (2005)
If a Tree Falls: A Story of the Earth Liberation Front (2011)
Inconvenient Truth, An (2006)
Island of the Blue Dolphins (1964)
Mia and the Migoo (2008)
Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)
No Impact Man: The Documentary (2009)
Planet Earth (2006)
Planet Earth II (2016)
Salt and Fire (2016)
Simpsons Movie, The (2007)
Surviving Progress (2011)
Torrente 3: El protector (2005)
WALL·E (2008)
When Elephants Were Young (2016)


In [39]:
def q3(input_title: str):
    # TODO: 
    movid_cursor = db.movies.find_one({'title': input_title}, {'movieId': 1, '_id':0})    
    if not movid_cursor:
        print(f"No movie found with title: {input_title}")
        return        
    mov_id = movid_cursor['movieId'] 
    
    
    ratings_cursor = db.ratings.find({ 'movieId': mov_id}, {"_id": 0, "rating": 1})
    
    sum_rating = 0
    count_rating = 0 
    
    for rating in ratings_cursor:
        sum_rating += rating['rating']
        count_rating += 1
    
    if not count_rating:
        print(f"No ratings found for movie: {input_title}")
        return                 

    _avg = sum_rating/count_rating
    print('{:.3f}'.format(_avg))


q3('Doom Generation, The (1995)')


# if __name__ == '__main__':
#     q3(argv[1])


No ratings found for movie: Doom Generation, The (1995)


In [41]:
from pymongo import MongoClient

def get_average_rating(movie_title):
    # MongoDB 서버에 연결
    client = MongoClient('mongodb://127.0.0.1:27017/')
    db = client['movielens']
    
    # movies 컬렉션에서 입력받은 영화 제목의 movieId 찾기
    movie = db.movies.find_one({"title": movie_title}, {"_id": 0, "movieId": 1})
    
    # 해당 영화 제목이 존재하지 않는 경우
    if not movie:
        print(f"No movie found with title: {movie_title}")
        return
    
    # ratings 컬렉션에서 해당 movieId의 모든 평점 조회
    ratings_cursor = db.ratings.find({"movieId": movie['movieId']}, {"_id": 0, "rating": 1})
    
    # 평점들의 평균 계산
    total_rating = 0
    count = 0
    for rating in ratings_cursor:
        total_rating += rating['rating']
        count += 1
    
    # 해당 영화에 대한 평점이 없는 경우
    if count == 0:
        print(f"No ratings found for movie: {movie_title}")
        return

    average_rating = total_rating / count
    print(f"Average rating for '{movie_title}': {average_rating:.2f}")

# 함수 호출 예시
get_average_rating('Doom Generation, The (1995)')


Average rating for 'Doom Generation, The (1995)': 2.84


In [254]:
# 각 질문에 대한 쿼리 및 인덱스 생성
queries = [
    # Question #2: 인덱스 생성 전
    ("Question #2: 인덱스 생성 전", lambda: db.ml_tags.find({'tag': 'American Civil War'}).sort('title', 1).distinct('title'), None),

    # 인덱스 생성 (1번째 인덱스: 'tag' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_tags, [('tag', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'title' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_movies, [('movidId', 1)]), None),

    # Question #2: 인덱스 생성 후 (3번째 인덱스: 'title' 필드)
    ("Question #2: 인덱스 생성 후", lambda: db.ml_tags.find({'tag': 'American Civil War'}).sort('title', 1).distinct('title'), [('title', 1)]),

    # 인덱스 삭제
    ("인덱스 삭제", lambda: drop_indexes(db), None),

    # Question #3: 인덱스 생성 전
    ("Question #3: 인덱스 생성 전", lambda: db.ml_ratings.aggregate([
        {'$match': {'title': 'Abraham Lincoln: Vampire Hunter (2012)'}},
        {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
    ]), None),

    # 인덱스 생성 (1번째 인덱스: 'title' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_movies, [('title', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'rating' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_ratings, [('movidId', 1)]), None),

    # Question #3: 인덱스 생성 후 (3번째 인덱스: 'rating' 필드)
    ("Question #3: 인덱스 생성 후", lambda: db.ml_ratings.aggregate([
        {'$match': {'title': 'Abraham Lincoln: Vampire Hunter (2012)'}},
        {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
    ]), [('rating', 1)]),

    # 인덱스 삭제
    ("인덱스 삭제", lambda: drop_indexes(db), None),

    # Question #4: 인덱스 생성 전
    ("Question #4: 인덱스 생성 전", lambda: db.ml_ratings.aggregate([
        {'$match': {'userId': 2}},
        {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
        {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
    ]), None),

    # 인덱스 생성 (1번째 인덱스: 'userId' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_ratings, [('userId', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'rating' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_ratings, [('movieId', 1)]), None),

    # Question #4: 인덱스 생성 후 (3번째 인덱스: 'rating' 필드)
    ("Question #4: 인덱스 생성 후", lambda: db.ml_ratings.aggregate([
        {'$match': {'userId': 2}},
        {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
        {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
    ]), [('rating', 1)]),
]

# 결과 저장할 딕셔너리
results = {}

# 10000번 반복
for _ in range(10000):
    # 각 쿼리를 차례대로 실행하고 실행 시간 출력
    for i in range(0, len(queries), 5):  # 인덱스 추가 후 인덱스 생성 후 질문까지 처리하는 부분을 추가하여 index를 5씩 증가시킴
        query_name = queries[i][0]
        before_query_func = queries[i+1][1]
        after_query_func = queries[i+3][1]  # 수정된 부분: 인덱스 생성 후 쿼리 함수의 인덱스를 올바르게 설정

        # 인덱스 생성 전 쿼리 실행 및 시간 측정
        before_execution_time = measure_execution_time(before_query_func)

        # 인덱스 생성
        create_index(db[query_name], queries[i+1][2])

        # 인덱스 추가
        create_index(db[query_name], queries[i+2][2])

        # 인덱스 생성 후 쿼리 실행 및 시간 측정
        after_execution_time = measure_execution_time(after_query_func)

        # 차이 계산
        diff = round(before_execution_time - after_execution_time, 3)  # 수정된 부분: 생성 전 시간에서 생성 후 시간을 빼서 차이를 계산

        # 결과 저장
        if query_name not in results:
            results[query_name] = {"생성 전": [], "생성 후": [], "차이": []}

        results[query_name]["생성 전"].append(before_execution_time)
        results[query_name]["생성 후"].append(after_execution_time)
        results[query_name]["차이"].append(diff)

# 평균값 계산 및 결과 출력
avg_diff = []
for query_name, result in results.items():
    before_time = 0
    after_time = 0
    print(query_name.split(':')[0] + ":")
    for key, value in result.items():
        if key == "차이":
            print(f"{key}: {round(before_time-after_time, 3)}ms")
            avg_diff.append(before_time-after_time)
        elif key == "생성 전":
            print(f"{key}: {value[0]:.3f}ms")
            before_time = value[0]
        elif key == "생성 후":
            print(f"{key}: {value[0]:.3f}ms")
            after_time = value[0]
    print()

# 각 question의 차이 값 출력
average_diff_mean = {f"#{i+2}의 차이": round(sum(result['차이']) / len(result['차이']), 3) for i, result in enumerate(results.values())}
print(", ".join(average_diff_mean.keys()) + " 평균:", round(sum(avg_diff) / len(avg_diff), 3), "ms")


Question #2:
생성 전: 1.001ms
생성 후: 1.013ms
차이: -0.012ms

Question #3:
생성 전: 0.983ms
생성 후: 0.000ms
차이: 0.983ms

Question #4:
생성 전: 0.000ms
생성 후: 0.999ms
차이: -0.999ms

#2의 차이, #3의 차이, #4의 차이 평균: -0.009 ms


In [ ]:
# 결과 저장할 딕셔너리
results = {}

# 1000번 반복
for _ in range(1000):
    # 각 쿼리를 차례대로 실행하고 실행 시간 출력
    for i in range(0, len(queries), 5):  # 인덱스 추가 후 인덱스 생성 후 질문까지 처리하는 부분을 추가하여 index를 5씩 증가시킴
        query_name = queries[i][0]
        before_query_func = queries[i+1][1]
        after_query_func = queries[i+3][1]  # 수정된 부분: 인덱스 생성 후 쿼리 함수의 인덱스를 올바르게 설정

        # 인덱스 생성 전 쿼리 실행 및 시간 측정
        before_execution_time = measure_execution_time(before_query_func)

        # 인덱스 생성
        create_index(db[query_name], queries[i+1][2])

        # 인덱스 추가
        create_index(db[query_name], queries[i+2][2])

        # 인덱스 생성 후 쿼리 실행 및 시간 측정
        after_execution_time = measure_execution_time(after_query_func)

        # 차이 계산
        diff = round(before_execution_time - after_execution_time, 3)  # 수정된 부분: 생성 전 시간에서 생성 후 시간을 빼서 차이를 계산

        # 결과 저장
        if query_name not in results:
            results[query_name] = {"생성 전": [], "생성 후": [], "차이": []}

        results[query_name]["생성 전"].append(before_execution_time)
        results[query_name]["생성 후"].append(after_execution_time)
        results[query_name]["차이"].append(diff)




# 평균값 계산 및 결과 출력
for query_name, result in results.items():
    before_time = 0
    after_time = 0
    print(query_name.split(':')[0] + ":")
    for key, value in result.items():
        if key == "차이":
            print(f"{key}: {round(before_time-after_time, 3)}ms")
        elif key == "생성 전":
            print(f"{key}: {value[0]:.3f}ms")
            before_time = value[0]
        elif key == "생성 후":
            print(f"{key}: {value[0]:.3f}ms")
            after_time = value[0]
    print()

# 각 question의 차이 값 출력
average_diff_mean = {f"#{i+2}의 차이": round(sum(result['차이']) / len(result['차이']), 3) for i, result in enumerate(results.values())}
print(", ".join(average_diff_mean.keys()) + " 평균:", round(sum(average_diff_mean.values()) / len(average_diff_mean), 3), "ms")


In [7]:
newjeans.delete_many({})

DeleteResult({'n': 5, 'ok': 1.0}, acknowledged=True)

In [8]:
newjeans.insert_one({'name':'Minji', 'age':18})

InsertOneResult(ObjectId('660952d1aa6e97be12e47d1c'), acknowledged=True)

In [9]:
pprint(list(newjeans.find()))

Pretty printing has been turned OFF


In [89]:
newjeans.insert_many([
    {
        'name':'Hanni',
        'age': 19
    },
    {
        'name': 'Danielle',
        'age': 18
    },
    {
        'name': 'Haerin',
        'age': 17
    }
])

In [90]:
pprint(list(newjeans.find()))

[{'_id': ObjectId('6603ce812498dea9061d070f'), 'age': 18, 'name': 'Minji'},
 {'_id': ObjectId('6603ce822498dea9061d0710'), 'age': 19, 'name': 'Hanni'},
 {'_id': ObjectId('6603ce822498dea9061d0711'), 'age': 18, 'name': 'Danielle'},
 {'_id': ObjectId('6603ce822498dea9061d0712'), 'age': 17, 'name': 'Haerin'}]


In [91]:
newjeans.update_one(
    { 'name': 'Hannie' },
    { '$set': { 'location' : 'Rome' } },
    upsert=True
)

In [92]:
pprint(list(newjeans.find()))

[{'_id': ObjectId('6603ce812498dea9061d070f'), 'age': 18, 'name': 'Minji'},
 {'_id': ObjectId('6603ce822498dea9061d0710'), 'age': 19, 'name': 'Hanni'},
 {'_id': ObjectId('6603ce822498dea9061d0711'), 'age': 18, 'name': 'Danielle'},
 {'_id': ObjectId('6603ce822498dea9061d0712'), 'age': 17, 'name': 'Haerin'},
 {'_id': ObjectId('6603ce84fbd056ad72228dd2'),
  'location': 'Rome',
  'name': 'Hannie'}]


In [93]:
newjeans.update_many(
    { 'age': 19 },
    { '$set': { 'location' : 'Paris' } },
    upsert=True
)

In [94]:
pprint(list(newjeans.find()))

[{'_id': ObjectId('6603ce812498dea9061d070f'), 'age': 18, 'name': 'Minji'},
 {'_id': ObjectId('6603ce822498dea9061d0710'),
  'age': 19,
  'location': 'Paris',
  'name': 'Hanni'},
 {'_id': ObjectId('6603ce822498dea9061d0711'), 'age': 18, 'name': 'Danielle'},
 {'_id': ObjectId('6603ce822498dea9061d0712'), 'age': 17, 'name': 'Haerin'},
 {'_id': ObjectId('6603ce84fbd056ad72228dd2'),
  'location': 'Rome',
  'name': 'Hannie'}]


In [95]:
newjeans.update_one(
    { 'name': 'Hyein' },
    { '$set': { 'age' : '15' } },
    upsert=True
)

In [96]:
pprint(list(newjeans.find()))

[{'_id': ObjectId('6603ce812498dea9061d070f'), 'age': 18, 'name': 'Minji'},
 {'_id': ObjectId('6603ce822498dea9061d0710'),
  'age': 19,
  'location': 'Paris',
  'name': 'Hanni'},
 {'_id': ObjectId('6603ce822498dea9061d0711'), 'age': 18, 'name': 'Danielle'},
 {'_id': ObjectId('6603ce822498dea9061d0712'), 'age': 17, 'name': 'Haerin'},
 {'_id': ObjectId('6603ce84fbd056ad72228dd2'),
  'location': 'Rome',
  'name': 'Hannie'},
 {'_id': ObjectId('6603ce89fbd056ad7222900e'), 'age': '15', 'name': 'Hyein'}]


In [98]:
newjeans.delete_many({'location':'Paris'})

In [99]:
pprint(list(newjeans.find()))

[{'_id': ObjectId('6603ce812498dea9061d070f'), 'age': 18, 'name': 'Minji'},
 {'_id': ObjectId('6603ce822498dea9061d0711'), 'age': 18, 'name': 'Danielle'},
 {'_id': ObjectId('6603ce822498dea9061d0712'), 'age': 17, 'name': 'Haerin'},
 {'_id': ObjectId('6603ce84fbd056ad72228dd2'),
  'location': 'Rome',
  'name': 'Hannie'},
 {'_id': ObjectId('6603ce89fbd056ad7222900e'), 'age': '15', 'name': 'Hyein'}]


In [113]:
db = client.lab  
col = db.imdb

In [114]:
def input_movie_info():
    title = input("Movie title: ")
    director = input("Director: ")
    genre = input("Genre: ").split(",")
    score = float(input("Score: "))

    return title, director, genre, score

In [116]:
t, d, g, s = input_movie_info()

col.insert_one(
    {
        'title':t,
        'director': d,
        'genre': g,
        'score': s
    }
)

print("[INFO] Success to insert!")

[INFO] Success to insert!


In [112]:
pprint(list(col.find()))

[{'_id': ObjectId('6603d1f92498dea9061d0713'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'}]


In [117]:
def input_movie_info():
    title = input("Movie title: ")
    director = input("Director: ")
    genre = input("Genre: ").split(",")
    score = float(input("Score: "))

    return title, director, genre, score

In [118]:
quit = False
movies = []

while(not quit):
    t, d, g, s = input_movie_info()
    doc = {'title': t, 'director': d, 'genre': g, 'score': s }
    movies.append(doc)

    end = input("Quit? (Y/N)")

    if end =="Y":
        quit = True

col.insert_many(movies) 

print("[INFO] Success to insert!")

In [119]:
pprint(list(col.find()))

[{'_id': ObjectId('6603d1f92498dea9061d0713'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d26d2498dea9061d0714'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d2a12498dea9061d0715'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d43d2498dea9061d0716'),
  'director': 'Anthony Russo and Joe Russo',
  'genre': ['Action', ' Adventure', ' Drama'],
  'score': 8.6,
  'title': ' Avengers: Infinity War'},
 {'_id': ObjectId('6603d43d2498dea9061d0717'),
  'director': 'Christopher Landon',
  'genre': ['Drama', ' Horror', ' Mystery'],
  'score': 6.6,
  'title': 'Happy Death Day 2U'},
 {'_id': ObjectId('6603d43d2498dea9061d0718'),
  'director': 'Park Chan-Wook',
  'genre': ['Action', ' Drama', ' Mystery', ' Thriller'],
  'score': 8.4,
  'title': 'Oldboy'

In [122]:
title = input("title")
director = input("director")
genre = input("genre").split(",")
score = float(input("score"))

col.update_one({'Movie Title':title}, {'$set' : {'Director':director, 'Genre': genre, 'score': score}})

In [123]:
pprint(list(col.find()))

[{'_id': ObjectId('6603d1f92498dea9061d0713'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d26d2498dea9061d0714'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d2a12498dea9061d0715'),
  'director': 'Craig Gillespie',
  'genre': ['Comedy', ' Crime'],
  'score': 7.4,
  'title': 'Cruella'},
 {'_id': ObjectId('6603d43d2498dea9061d0716'),
  'director': 'Anthony Russo and Joe Russo',
  'genre': ['Action', ' Adventure', ' Drama'],
  'score': 8.6,
  'title': ' Avengers: Infinity War'},
 {'_id': ObjectId('6603d43d2498dea9061d0717'),
  'director': 'Christopher Landon',
  'genre': ['Drama', ' Horror', ' Mystery'],
  'score': 6.6,
  'title': 'Happy Death Day 2U'},
 {'_id': ObjectId('6603d43d2498dea9061d0718'),
  'director': 'Park Chan-Wook',
  'genre': ['Action', ' Drama', ' Mystery', ' Thriller'],
  'score': 8.4,
  'title': 'Oldboy'

In [130]:
print("[INFO] Enter the movie title you want to update")

t, d, g, s = input_movie_info()

col.update_one(
    {
        'title':t
    },
    {
        '$set': {
            'director': d,
            'genre': g,
            'score': s
        }
    }
)

print("[INFO] Success to update!")

[INFO] Enter the movie title you want to update
[INFO] Success to update!


In [131]:
pprint(list(col.find()))

[{'_id': ObjectId('6603d43d2498dea9061d0716'),
  'director': 'null',
  'genre': ['Genre_1', ' Genre_2'],
  'score': 6.6,
  'title': ' Avengers: Infinity War'},
 {'_id': ObjectId('6603d43d2498dea9061d0717'),
  'director': 'Christopher Landon',
  'genre': ['Drama', ' Horror', ' Mystery'],
  'score': 6.6,
  'title': 'Happy Death Day 2U'},
 {'_id': ObjectId('6603d43d2498dea9061d0718'),
  'director': 'Park Chan-Wook',
  'genre': ['Action', ' Drama', ' Mystery', ' Thriller'],
  'score': 8.4,
  'title': 'Oldboy'}]


In [128]:
print("[INFO] Enter the movie title you want to delete.")
title = input("Movie title: ")
delete_op = input("Are you sure? (Y/N)")

if delete_op == "Y":
    col.delete_many({
        'title': title
    })
    print("[INFO] Deleted")

[INFO] Enter the movie title you want to delete.
[INFO] Deleted


In [129]:
pprint(list(col.find()))

[{'_id': ObjectId('6603d43d2498dea9061d0716'),
  'director': 'Anthony Russo and Joe Russo',
  'genre': ['Action', ' Adventure', ' Drama'],
  'score': 8.6,
  'title': ' Avengers: Infinity War'},
 {'_id': ObjectId('6603d43d2498dea9061d0717'),
  'director': 'Christopher Landon',
  'genre': ['Drama', ' Horror', ' Mystery'],
  'score': 6.6,
  'title': 'Happy Death Day 2U'},
 {'_id': ObjectId('6603d43d2498dea9061d0718'),
  'director': 'Park Chan-Wook',
  'genre': ['Action', ' Drama', ' Mystery', ' Thriller'],
  'score': 8.4,
  'title': 'Oldboy'}]
